In [ ]:
!pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv
!pip install sentence-transformers
!pip install langchain
!pip install pypdf
!pip install chromadb
!pip install -U langchain-community
!pip install --upgrade chromadb

In [ ]:
import google.generativeai as genai
from google.colab import userdata
from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown
import textwrap

In [ ]:
def to_markdown(text):
  text = text.replace('•','*')
  return Markdown(textwrap.indent(text, '>', predicate=lambda _: True))

In [ ]:
# A simple request to Google Gemini by Langchain
from langchain_google_genai import ChatGoogleGenerativeAI
GOOGLE_API_KEY = "___"
llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY)
result = llm.invoke("bệnh ung thư là gì?")
to_markdown(result.content)

In [ ]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
!mkdir pdfs
# !gdown /content/mot so cay thuoc viet nam theo nhom benh.pdf
# !gdown 1vILwiv6nS2wI3chxNabMgry3qnV67TxM -O pdfs/rachelgreecv.pdf

In [ ]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/DataSet/__")
data = loader.load_and_split()

In [ ]:
context = "\n".join(str(p.page_content) for p in data)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
context = "\n\n".join(str(p.page_content) for p in data)
texts = text_splitter.split_text(context)
print(len(texts))

In [ ]:
# Sử dụng mô hình embedding
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
MODEL_NAME = "keepitreal/vietnamese-sbert"
hf_embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

In [ ]:
vie_db = Chroma.from_texts(texts, hf_embeddings, persist_directory="db")

In [ ]:
question = "Công dụng của cây đinh lăng"
docs = vie_db.similarity_search(question, k =2)

for doc in docs:
  print("---------------")
  print(doc.page_content)

In [ ]:
type(docs)

In [ ]:
prompt_template = """

Trả lời câu hỏi một cách cụ thể nhất có thể từ ngữ cảnh được cung cấp, đảm bảo cung cấp tất cả các chi tiết,
nếu câu trả lời không có trong ngữ cảnh đã cung cấp, chỉ cần nói "câu trả lời không có sẵn trong ngữ cảnh", đừng cung cấp câu trả lời sai.\n\n
ngữ cảnh:\n {context}?\n
câu hỏi: \n {question}\n

câu trả lời:

"""

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key = GOOGLE_API_KEY,
                             temperature=0.3)


In [ ]:
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [ ]:
question = "Công dụng của cây đinh lăng ?"
response = chain(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)


print(response)

In [ ]:
# sentence embeding
from sentence_transformers import SentenceTransformer
sentences = ["Cô giáo đang ăn kem", "Chị gái đang thử món thịt dê"]
vie_model = SentenceTransformer('keepitreal/vietnamese-sbert')
vie_embeddings = vie_model.encode(sentences)
print(vie_embeddings)